In [36]:
import os
from pinecone import Pinecone, ServerlessSpec
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone as PineconeVectorStore
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import ollama
from dotenv import load_dotenv

In [37]:
# Load environment variables (replace or ensure .env has these)
PINECONE_API_KEY = "pcsk_5WhKTT_4whYJxqvTNsVyEHVR73WfmzWpwnVDVdFhS9u1PSgQkG9Vwd7tVr53t7j7UzPYLB"  # Replace with your Pinecone API key
PINECONE_ENVIRONMENT = "us-east-1"  # Replace with your Pinecone environment
PINECONE_INDEX_NAME = "medical-chatbot"  # Your Pinecone index name

# Step 1: Initialize Pinecone using the new API
pinecone_instance = Pinecone(api_key=PINECONE_API_KEY)
serverless_spec = ServerlessSpec(region=PINECONE_ENVIRONMENT, cloud="aws")

In [38]:
# Step 2: Create or connect to the Pinecone index
existing_indexes=[index['name'] for index in pinecone_instance.list_indexes()]
if PINECONE_INDEX_NAME not in existing_indexes:
        pinecone_instance.create_index(
            name=PINECONE_INDEX_NAME,
            dimension=384,
            metric="cosine",
            spec=serverless_spec
        )
index = pinecone_instance.Index(PINECONE_INDEX_NAME)
print(f"Connected to Pinecone index: {PINECONE_INDEX_NAME}")

Connected to Pinecone index: medical-chatbot


In [39]:
# Step 3: Load and process the PDF file
loader = PyPDFLoader("medical_book.pdf")  # Replace with your PDF file
documents = loader.load()

In [56]:
# Split documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [57]:
# Step 4: Generate embeddings and upload them to Pinecone
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [58]:
import os
os.environ['PINECONE_API_KEY'] = "pcsk_5WhKTT_4whYJxqvTNsVyEHVR73WfmzWpwnVDVdFhS9u1PSgQkG9Vwd7tVr53t7j7UzPYLB"

In [59]:
# Convert documents to Pinecone-compatible vector store
vectorstore = PineconeVectorStore.from_texts(
    texts=[doc.page_content for doc in texts],
    embedding=embeddings,
    index_name=PINECONE_INDEX_NAME,
    namespace="default",  # Optional: Define a namespace
    # pinecone_client=pinecone_instance  # Pass the initialized Pinecone client
)
print("PDF content successfully uploaded to Pinecone.")

PDF content successfully uploaded to Pinecone.


In [61]:
from langchain_ollama import OllamaLLM

In [65]:
import ollama
from langchain_community.llms import Ollama

In [66]:
def query_chatbot(question):
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    docsearch = Pinecone.from_existing_index(index_name=PINECONE_INDEX_NAME,embedding=embeddings)
    llm = Ollama(model="llama3.2")
    qa_chain=RetrievalQA.from_chain_type(llm=llm, chain_type="map_reduce", retriever=docsearch.as_retriever())
    response=qa_chain.run(question)
    return response

In [67]:
# Step 6: Ask a question to the chatbot
query = "What are vitamins? Name some vitamins."
response = query_chatbot(query)
print("Chatbot's response:", response)

Chatbot's response: Based on the provided text, I don't have enough information to determine what vitamins are. The text does not provide a clear definition of vitamins or list specific types of vitamins.

However, it is known that there are two main categories of vitamins:

1. Water-soluble vitamins (also known as fat-soluble vitamins):
	* Vitamin A
	* Vitamin D
	* Vitamin E
	* Vitamin K
2. Fat-soluble vitamins:
	* Vitamins B

The text only provides a definition of water-soluble vitamins, which includes any information about what vitamins are.

Therefore, I must say that I don't know the specific answer to your question about vitamins.


In [68]:
# Step 6: Ask a question to the chatbot
query = "What is acne"
response = query_chatbot(query)
print("Chatbot's response:", response)

Chatbot's response: Based on the provided texts, "Acne" is not explicitly defined or explained in the given snippet. However, it is mentioned as a topic within the broader category of "Cutaneous Infections, Acne, and Other Common Skin Disorders."

The most relevant information about acne can be found in other sources, such as:

* Wolf Jr., John E. “Contact Dermatitis.” In Conn’s Current Therapy, 1996, ed. Robert E. Rakel. Philadelphia: W. B. Saunders Co., 1996.
* Anthony S. Fauci, et al. New York: McGraw-Hill, 1997.

These sources provide more comprehensive information about acne, but they are not included in the provided snippet.

Therefore, I do not have enough information to provide a definitive answer to what acne is based on the given texts.
